In [1]:
import pandas as pd
import numpy as np
import censusdata

In [13]:
#full_nation_tracts validate the row tallies from the above data frame with the count from 
# https://www.census.gov/geographies/reference-files/time-series/geo/tallies.html

# Census Blocks: 11,078,297
# My Count:      11,078,297 

# Census Tracts: 73,057
# My Count :     73,057

In [14]:
# group the blocks by state, county, and tract to aggregate population to the tract level

full_nation_tracts = full_nation_df.groupby(['STATE', 'COUNTY', 'TRACT'])['P001001'].sum().reset_index()

In [15]:
full_nation_tracts['P001001'].sum()

308745538

In [4]:
#Load in data

#there are only 49 states in the lower_state_districts file - Nebraska has a unicameral body, and is represented in the upper
#chamber file

lower_state_districts_csv = pd.read_csv('data/raw/National_2018SLDL.txt', dtype = 'str')

lower_state_districts = pd.DataFrame(data = lower_state_districts_csv)

upper_state_districts_csv = pd.read_csv('data/raw/National_2018SLDU.txt', dtype = 'str')

upper_state_districts = pd.DataFrame(data = upper_state_districts_csv)

FileNotFoundError: [Errno 2] No such file or directory: 'data/raw/National_2018SLDL.txt'

In [18]:
# this following section is based on the guide found here: 
# https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html

###Lower State District Mappings

##State

lower_state_districts['state_fip'] = lower_state_districts.BLOCKID.str[0:2] 

##County from 15 character GEOID

lower_state_districts['county_fip'] = lower_state_districts.BLOCKID.str[2:5] 

##County Subdivision from 15 character GEOID

lower_state_districts['census_tract'] = lower_state_districts.BLOCKID.str[5:11]

##Block Group from 15 character GEOID

lower_state_districts['block_group'] = lower_state_districts.BLOCKID.str[11]

##Block from 15 character GEOID

lower_state_districts['block'] = lower_state_districts.BLOCKID.str[11:15]

In [19]:
###Upper State District Mappings

##State from 15 character GEOID

upper_state_districts['state_fip'] = upper_state_districts.BLOCKID.str[0:2] 

##County from 15 character GEOID

upper_state_districts['county_fip'] = upper_state_districts.BLOCKID.str[2:5] 

##County Subdivision from 15 character GEOID

upper_state_districts['census_tract'] = upper_state_districts.BLOCKID.str[5:11]

##Block Group from 15 character GEOID

upper_state_districts['block_group'] = upper_state_districts.BLOCKID.str[11]

##Block from 15 character GEOID

upper_state_districts['block'] = upper_state_districts.BLOCKID.str[11:15]

In [20]:
full_nation_df

,STATE,COUNTY,TRACT,BLOCK,P001001
"Block 1000, Block Group 1, Census Tract 201, Autauga County, Alabama: Summary level: 101, state:01> county:001> tract:020100> block:1000",1,1,20100,1000,61
"Block 1001, Block Group 1, Census Tract 201, Autauga County, Alabama: Summary level: 101, state:01> county:001> tract:020100> block:1001",1,1,20100,1001,0
"Block 1002, Block Group 1, Census Tract 201, Autauga County, Alabama: Summary level: 101, state:01> county:001> tract:020100> block:1002",1,1,20100,1002,0
"Block 1003, Block Group 1, Census Tract 201, Autauga County, Alabama: Summary level: 101, state:01> county:001> tract:020100> block:1003",1,1,20100,1003,75
"Block 1004, Block Group 1, Census Tract 201, Autauga County, Alabama: Summary level: 101, state:01> county:001> tract:020100> block:1004",1,1,20100,1004,0
...,...,...,...,...,...
"Block 3053, Block Group 3, Census Tract 111, District of Columbia, District of Columbia: Summary level: 101, state:11> county:001> tract:011100> block:3053",11,1,11100,3053,0
"Block 3054, Block Group 3, Census Tract 111, District of Columbia, District of Columbia: Summary level: 101, state:11> county:001> tract:011100> block:3054",11,1,11100,3054,0
"Block 3055, Block Group 3, Census Tract 111, District of Columbia, District of Columbia: Summary level: 101, state:11> county:001> tract:011100> block:3055",11,1,11100,3055,0
"Block 3056, Block Group 3, Census Tract 111, District of Columbia, District of Columbia: Summary level: 101, state:11> county:001> tract:011100> block:3056",11,1,11100,3056,0


In [21]:
for i in lower_state_districts.columns:
    
    print((lower_state_districts[i] == 'ZZZ').sum())
          
          
          
          

0
5525
0
0
0
0
0


In [22]:
# Start by renaming columns common between two data sets so they match
# Make a new list of columns that are being used to merge
# Ensure join columns in both are ints using .astype()
# Join

In [26]:
#Rename columns for easier merging
JOIN_COLUMNS = ['state_fip', 'county_fip', 'census_tract', 'block']

full_nation_df.rename(columns = {'STATE': 'state_fip', 'COUNTY' : 'county_fip', 'TRACT' : 'census_tract', 'BLOCK' : 'block'}, inplace = True)

full_nation_df[JOIN_COLUMNS] = full_nation_df[JOIN_COLUMNS].astype(int)

lower_state_districts[JOIN_COLUMNS] = lower_state_districts[JOIN_COLUMNS].astype(int)

upper_state_districts[JOIN_COLUMNS] = upper_state_districts[JOIN_COLUMNS].astype(int)

In [52]:
lower_state_districts.rename(columns = {'DISTRICT' : 'lower_leg_district'}, inplace = True)

upper_state_districts.rename(columns = {'DISTRICT' : 'upper_leg_district'}, inplace = True)

lower_state_districts.drop(columns = ['BLOCKID', 'block_group'], inplace = True)

upper_state_districts.drop(columns = ['BLOCKID', 'block_group'], inplace = True)

In [54]:
##Join in state codes

full_nation_df = pd.merge(full_nation_df, lower_state_districts, how = 'outer', on = JOIN_COLUMNS)

full_nation_df = pd.merge(full_nation_df, upper_state_districts, how = 'outer', on = JOIN_COLUMNS)

In [56]:
full_nation_df

,state_fip,county_fip,census_tract,block,P001001,BLOCKID_x,lower_leg_district,block_group_x,BLOCKID_y,upper_leg_district,block_group_y
0,1,1,20100,1000,61,010010201001000,088,1,010010201001000,030,1
1,1,1,20100,1001,0,010010201001001,088,1,010010201001001,030,1
2,1,1,20100,1002,0,010010201001002,088,1,010010201001002,030,1
3,1,1,20100,1003,75,010010201001003,088,1,010010201001003,030,1
4,1,1,20100,1004,0,010010201001004,088,1,010010201001004,030,1
...,...,...,...,...,...,...,...,...,...,...,...
10878433,56,45,951300,3125,0,560459513003125,002,3,560459513003125,003,3
10878434,56,45,951300,3126,0,560459513003126,002,3,560459513003126,003,3
10878435,56,45,951300,3127,15,560459513003127,002,3,560459513003127,003,3
10878436,56,45,951300,3128,0,560459513003128,002,3,560459513003128,003,3


In [48]:
tract_to_puma = pd.read_csv('2010_Census_Tract_to_2010_PUMA.txt')

In [59]:
tract_to_puma.rename(columns = {'STATEFP' : 'state_fip', 'COUNTYFP' :'county_fip', 'TRACTCE':'census_tract', 'PUMA5CE' : 'puma'}, inplace = True)

COLUMNS = ['state_fip', 'county_fip', 'census_tract']


full_nation_df = tract_to_puma.merge(full_nation_df, on = COLUMNS, how = 'outer')

In [63]:
#rename population
#city, congressional district, acs pop @ puma level, adjusted block pop

#export state, county, tract, puma, block, population, lower, uppper
full_nation_df[['state_fip','county_fip','census_tract','puma','block','P001001','lower_leg_district','upper_leg_district']].to_csv('master_block.csv.gz', index = False, compression = 'gzip')